In [6]:
import os
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import numpy as np

In [2]:
class Model:
    def __init__(self):
        # Disable oneDNN optimizations warning
        os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

        # Model and Tokenizer Name
        model_name = "google-bert/bert-base-uncased"  # Replace with a valid model name

        # Load Tokenizer and Model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = TFAutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=3
        )

    def SetDatasets(self, texts, labels):
        # Example Dataset (Texts and Labels)
        self.texts = texts
        self.labels = labels

        self.train_data, self.train_labels = self.preprocess(self.texts, self.labels)

    def SplitDataset(self, testDataPercentage):
        pass

    def LoadModelNTokenizer(self, modelLocation="./trained"):
        # Load the model and tokenizer from the saved directory
        loaded_model = TFAutoModelForSequenceClassification.from_pretrained(
            modelLocation
        )
        loaded_tokenizer = AutoTokenizer.from_pretrained(modelLocation)
        return loaded_model, loaded_tokenizer

    def SaveModelNTokenizer(self, modelLocation="./trained"):
        # Save the Model and Tokenizer
        self.model.save_pretrained(modelLocation)
        self.tokenizer.save_pretrained(modelLocation)

    # Tokenize the Dataset
    def preprocess(self, texts, labels, max_length=128):
        encodings = self.tokenizer(
            texts,
            max_length=max_length,
            truncation=True,
            padding="max_length",
            return_tensors="tf",
        )
        return encodings, tf.convert_to_tensor(labels)

    def CompileModel(self):
        # Compile the Model (use modern loss function to avoid deprecation)
        optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        metric = tf.keras.metrics.SparseCategoricalAccuracy()

        self.model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    def TrainModel(self, batchSize=2, epochs=3):
        self.CompileModel()
        # Train the Model
        self.history = self.model.fit(
            self.train_data, self.train_labels, batch_size=batchSize, epochs=epochs
        )
    def plot_training_history(self):
        # Check if training history exists
        if not hasattr(self, 'history'):
            print("Model hasn't been trained yet!")
            return
        
        # Get history data
        history = self.history.history
        
        # Plot Training & Validation Accuracy
        plt.figure(figsize=(12, 6))

        # Accuracy Plot
        plt.subplot(1, 2, 1)
        plt.plot(history['sparse_categorical_accuracy'], label='Train Accuracy')
        plt.title('Train Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend(loc='best')

        # Loss Plot
        plt.subplot(1, 2, 2)
        plt.plot(history['loss'], label='Train Loss')
        plt.title('Train Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend(loc='best')

        # Show the plot
        plt.tight_layout()
        plt.show()

In [3]:
model = Model()

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [253]:
test_texts = ["This is amazing!", "I hate it."]
test_data, _ = model.preprocess(test_texts, [1, 0])
predictions = loaded_model.predict(test_data)
# Process and Print Predictions
predicted_classes = np.argmax(predictions.logits, axis=1)
print(f"Predictions: {predicted_classes}")

1/1 [==============================] - 2s 2s/step
Predictions: [1 0]


In [7]:
# Cell for loading datasets
csv_file = "./datasets/cleaned_split-data.csv"
df = pd.read_csv(csv_file)

data = Dataset.from_pandas(df) 

sentiments = list()
comments = data['Comments']
for i in data['Sentiment']:
    sen = i.lower()
    if sen == "positive":
        sentiments.append(1)
    elif sen == "negative":
        sentiments.append(0)
    elif sen == "neutral":
        sentiments.append(2)


comments_train, comments_test, sentiments_train, sentiments_test = train_test_split(comments, sentiments, test_size=0.2, random_state=42)

model.SetDatasets(texts=comments_train, labels=sentiments_train)


In [ ]:
model.TrainModel(epochs=5, batchSize=32)


Epoch 1/5


In [9]:
model.SaveModelNTokenizer()

In [10]:
model.plot_training_history()


Model hasn't been trained yet!
